In [ ]:
#-*-coding:utf-8-*-
import csv
import pandas as pd
import json
from pandas.core.frame import DataFrame
import fileinput

#存储我们需要的字段到csv文件中
pid = []
pubname = []
tit = []
dig = []
con = []
readNum = []
for line in fileinput.FileInput("wxPubAnal.json/posts.json",openhook=fileinput.hook_encoded("utf-8")):
    s = json.loads(line)
    if s.has_key('content'):
        if s['msgBiz'] =="MjM5MDc0NTY2OA==":
            pubname.append('洞见')
        elif s['msgBiz'] == "MzUxODM4OTYzMg==":
            pubname.append('清华小五爷园')
        elif s['msgBiz'] == "MzA4MjEyNTA5Mw==":
            pubname.append('Python开发者')
        elif s['msgBiz'] == "MzI1NDY5NDM3OQ==":
            pubname.append('凤凰WEEKLY')
        else:
            pubname.append('沃顿商业')
        pid.append(s['_id']['$oid'])
        tit.append(s['title'])
        dig.append(s['digest'])
        con.append(s['content'])
        if s.has_key('readNum'):
            readNum.append(s['readNum'])
        else:
            readNum.append(0)
        
dic = {"pid":pid,
        "pubname":pubname,
        "title":tit,
       "digest":dig,
       "content":con,
      "readNum":readNum}
df = DataFrame(dic)
df.to_csv(r'wxPubAnal.json/posts.csv',columns=['pid','pubname','title','digest','content','readNum'],index=False,sep=',', encoding='utf_8_sig')

In [ ]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')   
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
import os
from pprint import pprint
import pandas as pd
import jieba

df = pd.read_csv("wxPubAnal.json/posts.csv", encoding = "utf-8")

con = df['title'].str.cat(df['content'])

con.head()

def load_stopwords():
    f_stop = open('stop_words.txt', 'r')
    sw = [line.strip() for line in f_stop]
    f_stop.close()
    return sw

#分词+过滤停用词

def word_cut(text):
    text = str(text)
    seg = jieba.cut(text.strip())
    outstr = ""
    for word in seg:  
            if word not in stopwords:  
                if word != '\t':  
                    outstr += word  
                    outstr += " "  
    return outstr

stopwords = load_stopwords()

df["con"] = con

df["con_cutted"] = df.con.apply(word_cut)

df.con_cutted.head()

In [ ]:
import numpy as np

#每一个单词关联一个唯一的ID
docs = [ [word for word in df.con_cutted[i].split() ] for i in range(0,len(df)) ]
word_count_dict = Dictionary(docs)
#过滤高频低频词
word_count_dict.filter_extremes(no_below=5, no_above=0.5) 
#将文档表示成词袋向量
bag_of_words_corpus = [word_count_dict.doc2bow(perdoc) for perdoc in docs] 

#保存模型
model_name = "./model.lda"  
if os.path.exists(model_name):
    lda_model = gensim.models.LdaModel.load(model_name)  
    print("loaded from old")
else:
    # preprocess()  第一个参数为选用的文档向量，num_topics为主题个数，id2word可选是选用的字典，
    lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=80, id2word=word_count_dict)
    lda_model.save(model_name)  
    print("loaded from new")

In [ ]:
from pymongo import MongoClient
#结果写入数据库
conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn.wxPub

In [ ]:
#{pid:pid, theme: “主题1”, words: [{name:”xxx”, freq:123}], weight}

perdoc = db.perdoc
topic_num=3

for i in range(0,len(df)):
    doc = [ word for word in df.con_cutted[i].split() ]
    doc_dict = word_count_dict.doc2bow(doc)
    result=lda_model[doc_dict]
    result=sorted(result,key=lambda tup: -1 * tup[1])#排序，只取前三的主题
    idx = 1
    for topic in result:
        if idx>topic_num:
            break
        #print_topic(x,y) x是主题的id，y是打印该主题的前y个词，词是按权重排好序的
        dict = {}
        dict['pid'] = str(df['pid'][i])
        print("文章"+str(i+1))
        dict['theme'] = str("主题"+str(idx))
        print("主题"+str(idx))
        dict['weight'] = str(topic[1])
        s = str(lda_model.print_topic(topic[0], 30))
        freq_word = s.split(" + ")
        for j in range(0,len(freq_word)):
            fw = freq_word[j].split("*")
            if dict.has_key('words'):
                dict['words'].append({'name':str(fw[1]), 'freq':fw[0]})
            else:
                dict['words'] = [{'name':str(fw[1]), 'freq':fw[0]}]
        idx += 1
        #print(dict)
        result = perdoc.insert(dict)
        #print(result)
#print(db.perdoc.count())

In [ ]:
#{pid:待查询文章id, sid:匹配文章id, similarity:pid和sid两篇文章相似度}
#每篇文章的相似列表
sim = db.sim

from gensim import corpora, similarities
import traceback
# 用bag_of_words_corpus作为特征，训练tf_idf_model
tf_idf_model = gensim.models.TfidfModel(bag_of_words_corpus)

# 每篇文章在vsm上的tf-idf表示
corpus_tfidf = tf_idf_model[bag_of_words_corpus]

def similarity( i, query, dictionary, corpus_tfidf ):
    try:

        # 建立索引
        index = similarities.MatrixSimilarity(corpus_tfidf)

        # 在dictionary建立query的vsm_tf表示
        query_bow = dictionary.doc2bow( query.lower().split() )

        # 查询在n_topics维空间的表示
        query_lda = tf_idf_model[query_bow]

        # 计算相似度
        simi = index[query_lda]
        query_simi_list = []
        for idx, item in enumerate(simi):
            query_simi_list.append( {'pid':str(df['pid'][i]), 'sid':str(df['pid'][idx]), 'similarity':str(item)} )
        result = sim.insert(query_simi_list)
        
    except Exception,e:
        print traceback.print_exc()

for i in range(0,len(df)):
    print("相似权重矩阵：")
    similarity(i, str(df.con_cutted[i]), word_count_dict, corpus_tfidf )